In [2]:
import requests
import os
import json
import pandas as pd
from pandas import json_normalize

In [3]:
%env BEARER_TOKEN= AAAAAAAAAAAAAAAAAAAAAPKsMQEAAAAAzz%2BOfrKTbyWOASJmxt8BAuIGnx0%3Dikjh5IxQK1hTKtrHAvr4sVJxkCKPjOnzUkheU4IK9XCvwOtblS

env: BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAAPKsMQEAAAAAzz%2BOfrKTbyWOASJmxt8BAuIGnx0%3Dikjh5IxQK1hTKtrHAvr4sVJxkCKPjOnzUkheU4IK9XCvwOtblS


In [4]:
def auth():
    return os.environ.get("BEARER_TOKEN")


def create_url():
    return "https://api.twitter.com/2/tweets/sample/stream"

In [5]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


In [31]:
def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))
    return response.json()

In [26]:
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    #print(json.dumps(response.json()))

In [29]:
def set_rules(headers, delete, bearer_token):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "dog has:images", "tag": "dog pictures"},
        {"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))


In [87]:
def get_stream(headers, set, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at&expansions=author_id&user.fields=created_at", headers=headers, stream=True,
    )
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            #print(json.dumps(json_response, indent=4, sort_keys=True)) 
            #print(type(json_response['data']['created_at'])) #class str
            print(json_response['data']['created_at'] +"  "+ json_response['data']['text'])
            #dataframe = pd.DataFrame.from_dict(json_response['data'], orient="index")
            #dataFrame = json_normalize(json_response['data'])
            
            

In [10]:
def main():
    bearer_token = os.environ.get("BEARER_TOKEN")
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)


In [89]:
if __name__ == "__main__":
    main()

Exception: Cannot get stream (HTTP 429): {"title":"ConnectionException","detail":"This stream is currently at the maximum allowed connection limit.","connection_issue":"TooManyConnections","type":"https://api.twitter.com/2/problems/streaming-connection"}